## Libraries Used

In [1]:
from ftplib import FTP
import string
import os
import time

import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)


import boto3
# Get the service client 
client = boto3.client('s3')


## Setting FTP Connection

In [2]:
# from ftplib import FTP

# Domain name or server ip:
ftp = FTP('13.233.66.93')

ftp.login(user='test_user', passwd = 'rishabh')

'230 Logged on'

### Get all the file names

In [3]:
# Set the current directory on the server
ftp.cwd('/files/ftp')

# Return a list of file names
ftp_files = ftp.nlst()

# Printing
pp.pprint(ftp_files)

[   '9G9VYW23CU.csv',
    'Big-Data-Landscape-2017.pdf',
    'Big-Data-Landscape-2018.pdf',
    'Big-Data-Landscape-2019.pdf',
    'D029LRLIRA.csv',
    'DN7A49XY69.csv',
    'EDTMQD3VFB.csv',
    'file.txt',
    'GJNZEL7QS7.csv',
    'Measurement_info.csv',
    'Measurement_summary.csv',
    'MOCK_DATA.json',
    'N84NQPAZ5A.csv',
    'people.json',
    'SG75B3AMDD.csv',
    'WMM6GGSTIQ.csv']


### Get only CSV files 

In [4]:
def filter_names(n) :
    if n.endswith('csv') :
        return True
    else :
        return False

ftp_files = list(filter( filter_names, ftp_files ))

# Printing
pp.pprint(ftp_files)

[   '9G9VYW23CU.csv',
    'D029LRLIRA.csv',
    'DN7A49XY69.csv',
    'EDTMQD3VFB.csv',
    'GJNZEL7QS7.csv',
    'Measurement_info.csv',
    'Measurement_summary.csv',
    'N84NQPAZ5A.csv',
    'SG75B3AMDD.csv',
    'WMM6GGSTIQ.csv']


### Files which are already present in S3

In [5]:
try :
    # Returns some or all of the objects in a bucket
    s3_files = client.list_objects_v2(Bucket = 'rishabhsengar2611')
except :
    # if wrong bucket name is entered
    print("No such Bucket \n")

pp.pprint(s3_files)

{   'Contents': [   {   'ETag': '"90782d0821d06338747f7927e53e3d00"',
                        'Key': 'SG75B3AMDD.csv',
                        'LastModified': datetime.datetime(2020, 4, 3, 8, 56, 35, tzinfo=tzutc()),
                        'Size': 387,
                        'StorageClass': 'STANDARD'},
                    {   'ETag': '"fb05fa19d0981b560475a6753d3b3faa"',
                        'Key': 'WMM6GGSTIQ.csv',
                        'LastModified': datetime.datetime(2020, 4, 3, 8, 56, 35, tzinfo=tzutc()),
                        'Size': 277,
                        'StorageClass': 'STANDARD'},
                    {   'ETag': '"d3ee934d3c9219a2f5a9230551cbf182"',
                        'Key': 'file.txt',
                        'LastModified': datetime.datetime(2020, 4, 2, 18, 13, 29, tzinfo=tzutc()),
                        'Size': 319,
                        'StorageClass': 'STANDARD'},
                    {   'ETag': '"9bcac1874c0a07eef4be75ea99e2272e"',
              

In [6]:
# Dictionary Access to get the name of the first file
s3_files['Contents'][0]['Key']

'SG75B3AMDD.csv'

In [7]:
# Dictionary Access to get the size of the first file
s3_files['Contents'][0]['Size']

387

### Get S3 file names and there size

In [8]:
# Used to store the s3 files names
s3_files_names = []

# Used to store the sizes of files
s3_files_size = []

print("File Name \t\t Size\n")
# Print the files
try :
    for i in range( 0,len(s3_files['Contents']) ) :
        s3_files_names.append(s3_files['Contents'][i]['Key'])
        print(s3_files['Contents'][i]['Key'], end='\t\t')
        s3_files_size.append(s3_files['Contents'][i]['Size'])
        print(s3_files['Contents'][i]['Size'], end='\n')
        
except KeyError :
    print("Bucket is Empty")


File Name 		 Size

SG75B3AMDD.csv		387
WMM6GGSTIQ.csv		277
file.txt		319
people.json		1612897


### Remove already present file in the s3

In [9]:
# Already present files
s3_files_present = []

def filter_names(n) :
    if n in s3_files_names :
        s3_files_present.append(n)
        return False
    else :
        return True

ftp_files = list(filter( filter_names, ftp_files ))


print(" To be Added")
pp.pprint(ftp_files)
print("\n Already Present")
pp.pprint(s3_files_present)

 To be Added
[   '9G9VYW23CU.csv',
    'D029LRLIRA.csv',
    'DN7A49XY69.csv',
    'EDTMQD3VFB.csv',
    'GJNZEL7QS7.csv',
    'Measurement_info.csv',
    'Measurement_summary.csv',
    'N84NQPAZ5A.csv']

 Already Present
['SG75B3AMDD.csv', 'WMM6GGSTIQ.csv']


In [10]:
# Make the Dictionary of filename and Size
ftp_size = {}
for i in ftp_files :
    ftp_size[i] = ftp.size(i)

pp.pprint(ftp_size)

{   '9G9VYW23CU.csv': 354,
    'D029LRLIRA.csv': 341,
    'DN7A49XY69.csv': 467,
    'EDTMQD3VFB.csv': 237,
    'GJNZEL7QS7.csv': 316,
    'Measurement_info.csv': 124452984,
    'Measurement_summary.csv': 94076158,
    'N84NQPAZ5A.csv': 429}


## Adding files to S3

In [11]:
# Each part must be at least 5 MB in size
# 1024*1024*6 == 6MB
chunk_size = 6291456

In [12]:
# ftp.size(ftp_files[0])
for file in ftp_size :
    print(file+"\t\t"+str(ftp_size[file]))

9G9VYW23CU.csv		354
D029LRLIRA.csv		341
DN7A49XY69.csv		467
EDTMQD3VFB.csv		237
GJNZEL7QS7.csv		316
Measurement_info.csv		124452984
Measurement_summary.csv		94076158
N84NQPAZ5A.csv		429


### Files which have size less than 6MB uploaded

In [13]:
# Used to store the large files
big_files = []

# This for loop is used to upload files
for file in ftp_size :
    if ftp_size[file] < chunk_size :
        
        # Change the location
        local_file = os.path.join('/home/bluepi/Desktop/',file)
        
        # Retrieve a file in binary transfer mode
        # RETR command --> A RETR request asks the server to send the contents of a file 
        #                       over the data connection already established by the client.
        ftp.retrbinary('RETR ' + file, open(local_file, 'wb').write)
        print("retieved file :\t" + file,end='\n' )
        
        # Upload file in binary mode in s3 object
        # Useful when we perform multipsrt upload
        with open(local_file, 'rb') as data:
            client.upload_fileobj(Fileobj=data, Bucket= 'rishabhsengar2611', Key= file)
            
        print("uploaded file :\t" + file,end='\n')
        #os.remove(file)
    else :
        big_files.append(file)
        print("big files :  "+file+"  -->  size :  "+str(ftp_size[file]),end = '\n')
        
# Errors :-
# Brokenpipeerror errno 32
# ftplib.error_perm: 530 Login authentication failed

retieved file :	9G9VYW23CU.csv
uploaded file :	9G9VYW23CU.csv
retieved file :	D029LRLIRA.csv
uploaded file :	D029LRLIRA.csv
retieved file :	DN7A49XY69.csv
uploaded file :	DN7A49XY69.csv
retieved file :	EDTMQD3VFB.csv
uploaded file :	EDTMQD3VFB.csv
retieved file :	GJNZEL7QS7.csv
uploaded file :	GJNZEL7QS7.csv
big files :  Measurement_info.csv  -->  size :  124452984
big files :  Measurement_summary.csv  -->  size :  94076158
retieved file :	N84NQPAZ5A.csv
uploaded file :	N84NQPAZ5A.csv


### Files to be uploaded in chunks( Working on it )

In [14]:
big_files

['Measurement_info.csv', 'Measurement_summary.csv']

In [15]:
# # Add these files in s3

# # No of files present to add
# no_of_files = len(ftp_files)

# for i in range(0,no_of_files):

# # Get the filename
#     filename = ftp_files[i]

# #
#     localfile = open(filename, 'wb')
#     ftp.retrbinary('RETR ' + filename, localfile.write)
    
# #
# #     response = client.put_object(
# #     ACL='private',
# #     Body=filename, # Object data
# #     Bucket='rishabhsengar2611', # Bucket name to which the PUT operation was initiated
# #     Key=filename # Object key for which the PUT operation was initiated
# #     )
    



# Lambda Code "Same as above"

In [16]:
from ftplib import FTP
import string
import pprint
import time
import boto3
import os
client = boto3.client('s3')


def lambda_handler(event, context):
    
    try:

        ftp = FTP('13.233.66.93')
        ftp.login(user='test_user', passwd = 'rishabh')
        ftp.cwd('/files/ftp')
        ftp_files = ftp.nlst()
        #print(ftp_files,end='\n')
        
        
        def filter_names(n) :
            if n.endswith('csv') :
                return True
            else :
                return False

        ftp_files = list(filter( filter_names, ftp_files ))
        #print(ftp_files)
   
    except:
        print("Error connecting to FTP")
    
    try :
        s3_files = client.list_objects_v2(Bucket = 'rishabhsengar2611')
    except :
        print("No such Bucket \n")

# Returns some or all (up to 1,000) of the objects in a bucket
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(s3_files)

    
    
    s3_files_names = []
    s3_files_size = []
    #print("File Name \t Size\n")
    # Print the files
    try :
        for i in range( 0,len(s3_files['Contents']) ) :
            s3_files_names.append(s3_files['Contents'][i]['Key'])
            #print(s3_files['Contents'][i]['Key'], end='\t')
            s3_files_size.append(s3_files['Contents'][i]['Size'])
            #print(s3_files['Contents'][i]['Size'], end='\n')
        
    except KeyError :
        print("Bucket is Empty")


    
    def filter_names(n) :
        if n in s3_files_names :
            return False
        else :
            return True

    ftp_files = list(filter( filter_names, ftp_files ))
    #print(ftp_files)

    
    
    ftp_size = {}
    for i in ftp_files :
        ftp_size[i] = ftp.size(i)
    #print(ftp_size)



    chunk_size = 6291456
    
    
    
    big_files = []
    for file in ftp_size :
        if ftp_size[file] < chunk_size :
            localfile = os.path.join('/tmp/', file)
            
            ftp.retrbinary('RETR ' + file, open( localfile, 'wb').write)
            print("retieved file :\t" + file,end='\n' )
            
            with open(localfile, 'rb') as data:
                client.upload_fileobj(data, 'rishabhsengar2611', file)
            print("uploaded file :\t" + file,end='\n')
            #os.remove(file)
            
        else :
            big_files.append(file)
            print("big files :  "+file+"  -->  size :  "+str(ftp_size[file]),end = '\n')

    return "Success"
